In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import json
import re
import collections
import os
import datetime

In [2]:
mpd = []

In [89]:
def load_all(path):
    filenames = os.listdir(path)
    for filename in sorted(filenames):
        if filename.startswith("mpd.slice.") and filename.endswith(".json"):
            fullpath = os.sep.join((path, filename))
            f = open(fullpath)
            js = f.read()
            f.close()
            mpd_slice = json.loads(js)
            for playlist in mpd_slice['playlists']:
                mpd.append(playlist)

In [3]:
import sys
import json
import codecs
import datetime


pretty = True
compact = False
cache = {}

def get_playlist(pid):
    if pid >=0 and pid < 1000000:
        low = 1000 * int(pid / 1000)
        high = low + 999
        offset = pid - low
        path = "./mpd.v1/data/mpd.slice." + str(low) + '-' + str(high) + ".json"
        if not path in cache:
            f = codecs.open(path, 'r', 'utf-8')
            js = f.read()
            f.close()
            playlist = json.loads(js)
            cache[path] = playlist

        playlist = cache[path]['playlists'][offset]
        return playlist

def get_playlists_in_range(start, end):
    pls = []
    try:
        istart = int(start)
        iend = int(end)
        if istart <= iend and istart >= 0 and iend <= 1000000:
            for pid in xrange(istart, iend):
                pls.append(get_playlist(pid))
    except:
        raise
        print "bad pid"
    return pls

In [4]:
import sys
import json
import re
import collections
import os
import datetime

total_playlists = 0
total_tracks = 0
tracks = set()
artists = set()
albums = set()
titles = set()
total_descriptions = 0
ntitles = set()

playlist_length_histogram = collections.Counter()
num_followers_histogram = collections.Counter()
artist_histogram = collections.Counter()
track_histogram = collections.Counter()


def process_playlist(playlist):
    global total_playlists, total_tracks, total_descriptions

    total_playlists += 1
    # print playlist['playlist_id'], playlist['name']

    if 'description' in playlist:
        total_descriptions += 1

    titles.add(playlist['name'])
    nname = normalize_name(playlist['name'])
    ntitles.add(nname)

    playlist_length_histogram[playlist['num_tracks']] += 1
    num_followers_histogram[playlist['num_followers']] += 1

    for track in playlist['tracks']:
        total_tracks += 1
        albums.add(track['album_uri'])
        tracks.add(track['track_uri'])
        artists.add(track['artist_uri'])

        full_name = track['track_name'] + " by " + track['artist_name']
        artist_histogram[track['artist_name']] += 1
        track_histogram[full_name] += 1

def normalize_name(name):
    name = name.lower()
    name = re.sub(r"[.,\/#!$%\^\*;:{}=\_`~()@]", ' ', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

In [5]:
def show_summary():
    print
    print "number of playlists", total_playlists
    print "number of tracks", total_tracks
    print "number of unique tracks", len(tracks)
    print "number of unique albums", len(albums)
    print "number of unique artists", len(artists)
    print "number of unique titles", len(titles)
    print "number of playlists with descriptions", total_descriptions
    print "number of unique normalized titles", len(ntitles)
    print "avg playlist length", float(total_tracks) / total_playlists
    
    print
    print "top tracks"
    for track, count in track_histogram.most_common(20):
        print "%7d %s" % (count, track)

    print
    print "top artists"
    for artist, count in artist_histogram.most_common(20):
        print "%7d %s" % (count, artist)

    print
    print "playlist length histogram"
    for length, count in playlist_length_histogram.most_common(20):
        print "%7d %d" % (count, length)

    print
    print "num followers histogram"
    for followers, count in num_followers_histogram.most_common(20):
        print "%7d %d" % (count, followers)


In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


l = get_playlists_in_range(0, 1000)
l = np.array(l)
for plist in l:
    process_playlist(plist)

In [7]:
show_summary()


number of playlists 1000
number of tracks 67503
number of unique tracks 34443
number of unique albums 19261
number of unique artists 9754
number of unique titles 869
number of playlists with descriptions 20
number of unique normalized titles 754
avg playlist length 67.503

top tracks
     55 One Dance by Drake
     52 HUMBLE. by Kendrick Lamar
     50 Broccoli (feat. Lil Yachty) by DRAM
     46 Closer by The Chainsmokers
     44 Congratulations by Post Malone
     42 Don't Let Me Down by The Chainsmokers
     39 Roses by The Chainsmokers
     39 Bounce Back by Big Sean
     39 iSpy (feat. Lil Yachty) by KYLE
     39 Jumpman by Drake
     38 Bad and Boujee (feat. Lil Uzi Vert) by Migos
     38 Mask Off by Future
     37 XO TOUR Llif3 by Lil Uzi Vert
     36 White Iverson by Post Malone
     36 Panda by Desiigner
     35 Caroline by Aminé
     35 Sorry by Justin Bieber
     35 Never Be Like You by Flume
     35 Gold Digger by Kanye West
     35 goosebumps by Travis Scott

top artists
  

Collaborative Filtering

In [8]:
import numpy as np
from sklearn.model_selection import train_test_split
l_train, l_test = train_test_split(l)

In [19]:
len(l_train)

750

In [16]:
import copy

#recommend for one list
#go through training set, calculate score, stored in list
#return top n ranked list sorted by score
def calc_score(playlist):
    dic = {}
    tracks = playlist['tracks']
    nname_test = []
    for track in tracks:
        nname = normalize_name(track['track_name'])
        nname_test.append(nname)
    total = len(nname_test)
    
    #for each playlist in training set, calculate score and add in result
    for pl in l_train:
        count = 0
        tracks = pl['tracks']
        for track in tracks:
            nname = normalize_name(track['track_name'])
            if nname in nname_test:
                count+=1
        #after finishing counting
        dic[pl['pid']] = float(count)/total
        
    return dic, nname_test

In [41]:
import operator
dic, names = calc_score(l_test[0])
result = sorted(dic.items(), key=operator.itemgetter(1))
result.reverse()

In [51]:
result[0]

(360, 0.3)

In [52]:
def get_pid_tracks():
    pid_tracks = {}
    for pl in l_train:
        pid = pl['pid']
        tracks = pl['tracks']
        pid_tracks[pid] = tracks
    return pid_tracks

In [63]:
pt = get_pid_tracks()

In [77]:
def recommend(pt, scores, names, count):
    result = []
    for score in scores:
        pid = score[0]
        tracks = pt[pid]
        for track in tracks:
            nname = normalize_name(track['track_name'])
            if nname not in names:
                result.append(track)
                if len(result) >= count:
                    return result
    return result

In [90]:
z = recommend(pt, result, names, 10)

In [94]:
z

[{u'album_name': u'Ice On The Dune',
  u'album_uri': u'spotify:album:1olQuvonXXUEourYrj6daN',
  u'artist_name': u'Empire of the Sun',
  u'artist_uri': u'spotify:artist:67hb7towEyKvt5Z8Bx306c',
  u'duration_ms': 204591,
  u'pos': 0,
  u'track_name': u'Alive',
  u'track_uri': u'spotify:track:5WBnKCEsPwsvWHUZmLjS3s'},
 {u'album_name': u'Surf',
  u'album_uri': u'spotify:album:3eM1KTKmpqrQOvuvYY42cr',
  u'artist_name': u'Donnie Trumpet & The Social Experiment',
  u'artist_uri': u'spotify:artist:0ojcq9LJQWMawQdFDw3M0L',
  u'duration_ms': 226013,
  u'pos': 1,
  u'track_name': u'Sunday Candy',
  u'track_uri': u'spotify:track:6fTdcGsjxlAD9PSkoPaLMX'},
 {u'album_name': u'All My Friends (The Remixes)',
  u'album_uri': u'spotify:album:3SStFwKQqnQMtbMtpdO9Oc',
  u'artist_name': u'Snakehips',
  u'artist_uri': u'spotify:artist:2FwJwEswyIUAljqgjNSHgP',
  u'duration_ms': 355373,
  u'pos': 2,
  u'track_name': u'All My Friends - 99 Souls Remix',
  u'track_uri': u'spotify:track:1szfOuGK4RWHrsHdiMimym'},
 

In [93]:
for pl in l_test:
    dic, names = calc_score(pl)
    result = sorted(dic.items(), key=operator.itemgetter(1))
    result.reverse()
    print (recommend(pt, result, names, 5))
    print 

[{u'pos': 0, u'artist_name': u'Empire of the Sun', u'track_uri': u'spotify:track:5WBnKCEsPwsvWHUZmLjS3s', u'artist_uri': u'spotify:artist:67hb7towEyKvt5Z8Bx306c', u'track_name': u'Alive', u'album_uri': u'spotify:album:1olQuvonXXUEourYrj6daN', u'duration_ms': 204591, u'album_name': u'Ice On The Dune'}, {u'pos': 1, u'artist_name': u'Donnie Trumpet & The Social Experiment', u'track_uri': u'spotify:track:6fTdcGsjxlAD9PSkoPaLMX', u'artist_uri': u'spotify:artist:0ojcq9LJQWMawQdFDw3M0L', u'track_name': u'Sunday Candy', u'album_uri': u'spotify:album:3eM1KTKmpqrQOvuvYY42cr', u'duration_ms': 226013, u'album_name': u'Surf'}, {u'pos': 2, u'artist_name': u'Snakehips', u'track_uri': u'spotify:track:1szfOuGK4RWHrsHdiMimym', u'artist_uri': u'spotify:artist:2FwJwEswyIUAljqgjNSHgP', u'track_name': u'All My Friends - 99 Souls Remix', u'album_uri': u'spotify:album:3SStFwKQqnQMtbMtpdO9Oc', u'duration_ms': 355373, u'album_name': u'All My Friends (The Remixes)'}, {u'pos': 3, u'artist_name': u'Fat Joe', u'tra


[{u'pos': 0, u'artist_name': u'Calle 13', u'track_uri': u'spotify:track:5PbZ0HW1a482puzxOmtW1o', u'artist_uri': u'spotify:artist:0yNSzH5nZmHzeE2xn6Xshb', u'track_name': u'Perseguido', u'album_uri': u'spotify:album:1gDqOyL8NmU2LQPtFutRng', u'duration_ms': 284933, u'album_name': u'MultiViral'}, {u'pos': 1, u'artist_name': u'Calle 13', u'track_uri': u'spotify:track:3kNVYo6BJE9AENxzokM9YC', u'artist_uri': u'spotify:artist:0yNSzH5nZmHzeE2xn6Xshb', u'track_name': u'Muerte En Hawaii', u'album_uri': u'spotify:album:3yqcj5G3zkeXuWIx6cjQgx', u'duration_ms': 189866, u'album_name': u'Entren Los Que Quieran'}, {u'pos': 3, u'artist_name': u'Nicky Jam', u'track_uri': u'spotify:track:1PtL0YQRtq3qxDGUSwZPJ5', u'artist_uri': u'spotify:artist:1SupJlEpv7RS2tPNRaHViT', u'track_name': u'Travesuras', u'album_uri': u'spotify:album:4etANPc5BMELu5yx987Rpx', u'duration_ms': 195453, u'album_name': u'Nicky Jam Hits'}, {u'pos': 4, u'artist_name': u'Zion', u'track_uri': u'spotify:track:6FZCj8wIH1sDklyZH3DEUC', u'ar

[{u'pos': 0, u'artist_name': u'Fetty Wap', u'track_uri': u'spotify:track:2d8JP84HNLKhmd6IYOoupQ', u'artist_uri': u'spotify:artist:6PXS4YHDkKvl1wkIl4V8DL', u'track_name': u'Trap Queen', u'album_uri': u'spotify:album:2gKQvajkEEaDtkqJ8FJ4uw', u'duration_ms': 222093, u'album_name': u'Fetty Wap'}, {u'pos': 1, u'artist_name': u'The Weeknd', u'track_uri': u'spotify:track:7t5m6zRwIEBxmPt8lNoPPi', u'artist_uri': u'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ', u'track_name': u'Acquainted', u'album_uri': u'spotify:album:28ZKQMoNBB0etKXZ97G2SN', u'duration_ms': 348853, u'album_name': u'Beauty Behind The Madness'}, {u'pos': 2, u'artist_name': u'Drake', u'track_uri': u'spotify:track:27GmP9AWRs744SzKcpJsTZ', u'artist_uri': u'spotify:artist:3TVXtAsR1Inumwj472S9r4', u'track_name': u'Jumpman', u'album_uri': u'spotify:album:1ozpmkWcCHwsQ4QTnxOOdT', u'duration_ms': 205879, u'album_name': u'What A Time To Be Alive'}, {u'pos': 4, u'artist_name': u'Rihanna', u'track_uri': u'spotify:track:1Tt4sE4pXi57mTD1GCzsqm', u

[{u'pos': 0, u'artist_name': u'Avril Lavigne', u'track_uri': u'spotify:track:4omisSlTk6Dsq2iQD7MA07', u'artist_uri': u'spotify:artist:0p4nmQO2msCgU4IF37Wi3j', u'track_name': u'Sk8er Boi', u'album_uri': u'spotify:album:7h6XeTzy0SRXDrFJeA9gO7', u'duration_ms': 204000, u'album_name': u'Let Go'}, {u'pos': 1, u'artist_name': u'Fountains Of Wayne', u'track_uri': u'spotify:track:27L8sESb3KR79asDUBu8nW', u'artist_uri': u'spotify:artist:1pgtr4nhBQjp9oCUBPyYWh', u'track_name': u"Stacy's Mom", u'album_uri': u'spotify:album:6TZp52tXShLQbq8yNMxqNT', u'duration_ms': 197986, u'album_name': u'Welcome Interstate Managers'}, {u'pos': 3, u'artist_name': u'Bowling For Soup', u'track_uri': u'spotify:track:5oQcOu1omDykbIPSdSQQNJ', u'artist_uri': u'spotify:artist:5ND0mGcL9SKSjWIjPd0xIb', u'track_name': u'1985', u'album_uri': u'spotify:album:43jBR9MNJk5AUuB56M0pRF', u'duration_ms': 193000, u'album_name': u"A Hangover You Don't Deserve"}, {u'pos': 5, u'artist_name': u'Avril Lavigne', u'track_uri': u'spotify:tr

[{u'pos': 0, u'artist_name': u'Sia', u'track_uri': u'spotify:track:4E3afPSY5fUEelQS9ppL0e', u'artist_uri': u'spotify:artist:5WUlDfRSoLAfcVSX1WnrxN', u'track_name': u'Alive', u'album_uri': u'spotify:album:77jAfTh3KH9K2reMOmTgOh', u'duration_ms': 263520, u'album_name': u'This Is Acting'}, {u'pos': 1, u'artist_name': u'Zara Larsson', u'track_uri': u'spotify:track:1rIKgCH4H52lrvDcz50hS8', u'artist_uri': u'spotify:artist:1Xylc3o4UrD53lo9CvFvVg', u'track_name': u'Lush Life', u'album_uri': u'spotify:album:5YLRVHDVRw3QqWbeTGpC5B', u'duration_ms': 201122, u'album_name': u'So Good'}, {u'pos': 2, u'artist_name': u'Zara Larsson', u'track_uri': u'spotify:track:6hmhG1b4LEyNuashVvuIAo', u'artist_uri': u'spotify:artist:1Xylc3o4UrD53lo9CvFvVg', u'track_name': u'Never Forget You', u'album_uri': u'spotify:album:5YLRVHDVRw3QqWbeTGpC5B', u'duration_ms': 213427, u'album_name': u'So Good'}, {u'pos': 3, u'artist_name': u'Twenty One Pilots', u'track_uri': u'spotify:track:3CRDbSIZ4r5MsZ0YwxuEkn', u'artist_uri':

...

[{u'pos': 0, u'artist_name': u'CHVRCHES', u'track_uri': u'spotify:track:3PvzvltQZLIKk9YgoMP3Yq', u'artist_uri': u'spotify:artist:3CjlHNtplJyTf9npxaPl5w', u'track_name': u'The Mother We Share', u'album_uri': u'spotify:album:54IMJ8S0redmP6krSlKZLD', u'duration_ms': 190160, u'album_name': u'The Bones Of What You Believe'}, {u'pos': 1, u'artist_name': u'Future Islands', u'track_uri': u'spotify:track:41X9To0bQ4fpoSRri4NHiI', u'artist_uri': u'spotify:artist:1WvvwcQx0tj6NdDhZZ2zZz', u'track_name': u'Seasons (Waiting On You)', u'album_uri': u'spotify:album:1dKh4z5Aayt8FFDWjO5FDh', u'duration_ms': 226440, u'album_name': u'Singles'}, {u'pos': 2, u'artist_name': u'Cloud Nothings', u'track_uri': u'spotify:track:68rdZO4AcPakdi2zaS7kPI', u'artist_uri': u'spotify:artist:6rnbB5fuUuCSsspvFsxIpT', u'track_name': u"I'm Not Part of Me", u'album_uri': u'spotify:album:3NNSJt3gWSmPmnjCwZyLA5', u'duration_ms': 275308, u'album_name': u'Here and Nowhere Else'}, {u'pos': 3, u'artist_name': u'David Bowie', u'trac

[{u'pos': 1, u'artist_name': u'Sia', u'track_uri': u'spotify:track:27SdWb2rFzO6GWiYDBTD9j', u'artist_uri': u'spotify:artist:5WUlDfRSoLAfcVSX1WnrxN', u'track_name': u'Cheap Thrills', u'album_uri': u'spotify:album:77jAfTh3KH9K2reMOmTgOh', u'duration_ms': 211666, u'album_name': u'This Is Acting'}, {u'pos': 2, u'artist_name': u'Zara Larsson', u'track_uri': u'spotify:track:6hmhG1b4LEyNuashVvuIAo', u'artist_uri': u'spotify:artist:1Xylc3o4UrD53lo9CvFvVg', u'track_name': u'Never Forget You', u'album_uri': u'spotify:album:5YLRVHDVRw3QqWbeTGpC5B', u'duration_ms': 213427, u'album_name': u'So Good'}, {u'pos': 4, u'artist_name': u'Bebe Rexha', u'track_uri': u'spotify:track:4qULqxfCRhJAXVc75UupQB', u'artist_uri': u'spotify:artist:64M6ah0SkkRsnPGtGiRAbb', u'track_name': u'No Broken Hearts (feat. Nicki Minaj)', u'album_uri': u'spotify:album:11jAMg5CsXfhmwe0A8BIU2', u'duration_ms': 239991, u'album_name': u'No Broken Hearts (feat. Nicki Minaj)'}, {u'pos': 5, u'artist_name': u'Snakehips', u'track_uri': u

[{u'pos': 0, u'artist_name': u'Rihanna', u'track_uri': u'spotify:track:6KFaHC9G178beAp7P0Vi5S', u'artist_uri': u'spotify:artist:5pKCCKE2ajJHZ9KAiaK11H', u'track_name': u'Desperado', u'album_uri': u'spotify:album:3Q149ZH46Z0f3oDR7vlDYV', u'duration_ms': 186466, u'album_name': u'ANTI'}, {u'pos': 1, u'artist_name': u'Bebe Rexha', u'track_uri': u'spotify:track:4ZJPwET9Jrgpkqi4Vo3Yg8', u'artist_uri': u'spotify:artist:64M6ah0SkkRsnPGtGiRAbb', u'track_name': u'Bad Bitch (feat. Ty Dolla $ign)', u'album_uri': u'spotify:album:2rBHhp9tNShxTb529Hi5AS', u'duration_ms': 197253, u'album_name': u'All Your Fault: Pt. 1'}, {u'pos': 2, u'artist_name': u'Ayo & Teo', u'track_uri': u'spotify:track:0pSBuHjILhNEo55xK1zrRt', u'artist_uri': u'spotify:artist:27mFvqQj8KpjmdKIqcw1mG', u'track_name': u'Rolex', u'album_uri': u'spotify:album:1bHj528aZfYOKq0ve5AHMr', u'duration_ms': 238586, u'album_name': u'Rolex'}, {u'pos': 3, u'artist_name': u'PnB Rock', u'track_uri': u'spotify:track:2xbI8Vmyv3TkpTdywpPyNw', u'artis

[{u'pos': 1, u'artist_name': u'Earth, Wind & Fire', u'track_uri': u'spotify:track:1mqlc0vEP9mU1kZgTi6LIQ', u'artist_uri': u'spotify:artist:4QQgXkCYTt3BlENzhyNETg', u'track_name': u'September', u'album_uri': u'spotify:album:2dZn2ZVkJYFIyXsVmjyp9Y', u'duration_ms': 216520, u'album_name': u'Now, Then & Forever'}, {u'pos': 2, u'artist_name': u'The Rolling Stones', u'track_uri': u'spotify:track:1dv3ePjze9tPq2pk8eWJdR', u'artist_uri': u'spotify:artist:22bE4uQ6baNwSHPVcDxLCe', u'track_name': u'Gimme Shelter', u'album_uri': u'spotify:album:0c78nsgqX6VfniSNWIxwoD', u'duration_ms': 270773, u'album_name': u'Let It Bleed'}, {u'pos': 3, u'artist_name': u'Phil Collins', u'track_uri': u'spotify:track:18AXbzPzBS8Y3AkgSxzJPb', u'artist_uri': u'spotify:artist:4lxfqrEsLX6N1N4OCSkILp', u'track_name': u'In The Air Tonight - 2015 Remastered', u'album_uri': u'spotify:album:1cM3r0WQZWNkCpEbmFjLln', u'duration_ms': 336453, u'album_name': u'Face Value (Deluxe Editon)'}, {u'pos': 4, u'artist_name': u'The Rolling

[{u'pos': 0, u'artist_name': u'Rancid', u'track_uri': u'spotify:track:3prgCqodybVT758HNY2SXW', u'artist_uri': u'spotify:artist:6xTk3EK5T9UzudENVvu9YB', u'track_name': u'I Wanna Riot', u'album_uri': u'spotify:album:5G6FHjebFlPzZ3i5iOeeot', u'duration_ms': 191666, u'album_name': u'All The Moon Stompers'}, {u'pos': 1, u'artist_name': u'Rancid', u'track_uri': u'spotify:track:0LWkdUtorgGHr1bwT2C6tY', u'artist_uri': u'spotify:artist:6xTk3EK5T9UzudENVvu9YB', u'track_name': u'Roots Radical', u'album_uri': u'spotify:album:1HisV3ZKLs9It3KlGcPki5', u'duration_ms': 167506, u'album_name': u'...And Out Come The Wolves'}, {u'pos': 2, u'artist_name': u'Rancid', u'track_uri': u'spotify:track:41kg2RvdwBuIv0OPpvBS4v', u'artist_uri': u'spotify:artist:6xTk3EK5T9UzudENVvu9YB', u'track_name': u'Time Bomb', u'album_uri': u'spotify:album:1HisV3ZKLs9It3KlGcPki5', u'duration_ms': 144333, u'album_name': u'...And Out Come The Wolves'}, {u'pos': 3, u'artist_name': u'Rancid', u'track_uri': u'spotify:track:1pNlZQeoPn